In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
def prepare_dataset(paths):
    paths=paths[1:]
    labels = pd.read_csv('/kaggle/input/words-label/words.txt',sep='\n')[15:]

    idd=labels['#--- words.txt ---------------------------------------------------------------#'].apply(lambda x:(x.split()[0])).values
    lab = labels['#--- words.txt ---------------------------------------------------------------#'].apply(lambda x:(x.split()[-1]).lower()).values
    targets = dict(zip(idd,lab))
    
    characters = set((''.join(lab)).lower())
    word_ind = dict(zip(['<START>','<END>','<PAD>']+list(characters),range(len(characters)+3)))
    
    labell = [x.split('/')[-1][:-4] for x in paths]    
    return dict(zip(labell,paths)),targets,list(targets.keys())[1:],word_ind

In [3]:
import time
import os
import scipy 
import scipy.io
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFile
# fix bugs with loading png files
ImageFile.LOAD_TRUNCATED_IMAGES = True

import matplotlib.pyplot as plt
import PIL
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import datasets, models, transforms
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, Subset, random_split


In [25]:
class CaltechDataset(Dataset):
    def __init__(self, imgs,dict_labels,name_ind,word_ind,transforms=None):
        # path to the dataset
        self.dict_labels=dict_labels
        # transformation for image and targets
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = imgs
        self.name_ind=name_ind
        self.word_ind = word_ind
    def __getitem__(self, idx):
        if self.name_ind[idx] not in self.imgs or self.name_ind[idx] not in self.dict_labels :
            print(self.name_ind[idx])
        img = cv2.imread(self.imgs[self.name_ind[idx]])
        if img is None:
            img = np.zeros((125,250,3))
            print('bad img')
        else:
            img=img[:,:,::-1]
        targ = self.dict_labels[self.name_ind[idx]]
        
        res = []#[self.word_ind['<START>']]
        for x in targ:
            res.append(self.word_ind[x])
        res.append(self.word_ind['<END>'])
        sent_len = len(res)
        res+=[self.word_ind['<PAD>']]*(60-len(res))
        if self.transforms is not None:
            img= self.transforms(PIL.Image.fromarray(np.uint8(img)))
        return img, torch.tensor(res),targ,sent_len

    def __len__(self):
        '''this method return len of dataset'''
        return len(self.name_ind)

In [28]:
df.__len__()

115320

In [30]:
400*64

25600

In [31]:
size = (125,250)
trans=transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor() # converts to [0,1] interval
])

imgs,dict_labels,name_ind,word_ind = prepare_dataset(paths)

df = CaltechDataset(imgs,dict_labels,name_ind,word_ind,transforms=trans)


In [ ]:
for i,x in enumerate(df):
    if i==2:
        plt.imshow(x[0].permute(1,2,0))
        print(x[1][:x[3]],x[2],x[3])
        break   

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(3,64,5)
        self.conv1 = nn.Conv2d(64,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.conv3 = nn.Conv2d(64,64,3,padding=1)
        self.conv4 = nn.Conv2d(128,128,3,padding=1)
        self.conv5 = nn.Conv2d(128,128,3,padding=1)
        self.conv6 = nn.Conv2d(128,128,3,padding=1)
        self.conv7 = nn.Conv2d(256,256,3,padding=1)
        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(64)
        self.batch3 = nn.BatchNorm2d(64)
        self.batch4 = nn.BatchNorm2d(128)
        self.batch5 = nn.BatchNorm2d(128)
        self.batch6 = nn.BatchNorm2d(128)

        
        #-------------------
        self.init_h =nn.Linear(30*61,256)
        self.init_c=nn.Linear(30*61,256)
        self.before_lstm = nn.Linear(30*61*4,64)
        self.LSTM = nn.LSTM(64,256)
        self.output_layer = nn.Linear(256,55)
    def base(self,x):
        x0=F.max_pool2d(F.relu(self.conv0(x)),2)
        x1=self.batch1(F.relu(self.conv1(x0)))
        x2=self.batch2(F.relu(self.conv2(x1)))
        x3=self.batch3(F.relu(self.conv3(x2)))
        x4=F.max_pool2d(self.batch4(F.relu(self.conv4(torch.cat((x3,x1),dim=1)))),2)
        x5=self.batch5(F.relu(self.conv5(x4)))
        x6=self.batch6(F.relu(self.conv6(x5)))
        x7=F.relu(self.conv7(torch.cat((x6,x4),dim=1)))
        return x7
    def init_hidden(self,base_out):
        hidden = base_out.mean(dim=1).view(-1,30*61)
        h = self.init_h(hidden)  # (batch_size, decoder_dim)
        c = self.init_c(hidden)
        return h.unsqueeze(0), c.unsqueeze(0)
    def forward(self,x,chars,length):
        base_out=self.base(x)
        hidden = self.init_hidden(base_out)
        in_lstm=self.before_lstm(base_out.view(base_out.size(0),64,-1))
        hidden = self.LSTM(in_lstm.permute(1,0,2),hidden)
        out = F.log_softmax(self.output_layer(hidden[0]),dim=2)
        
        return out    

In [12]:
loader = DataLoader(df, batch_size=4,drop_last=True)
model = Net()
for x,y,w,e in loader:
    break

In [13]:
e

tensor([2, 5, 3, 5])

In [15]:
loss = torch.nn.CTCLoss()
loss(model(x,'',''),y, torch.full(size=(4,), fill_value=64, dtype=torch.long),e)

tensor(73.4230, grad_fn=<MeanBackward0>)

In [33]:

loader = DataLoader(df, batch_size=64,drop_last=True,shuffle=False)
epochs = 20

device = torch.device("cuda")
model = Net()
# state = torch.load('loc.pth')
# model.load_state_dict(state)
model = model.to(device)
# we optimizing parameters of our model, other parameters will not be optimised if exist  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_ctc = torch.nn.CTCLoss()

In [42]:
model=model.to(device)

In [43]:
model.train()
for e in range(epochs):
    t = 0
    loss_avg = 0
    
    for i, (img, target,word,lns) in enumerate(loader):
        img = img.to(device)
        target = target.to(device)
        lengths = lns.to(device)
        optimizer.zero_grad()
        #print(target_packed.data.size())
        outputs= model(img,target,2)    
        loss = loss_ctc(outputs,target, torch.full(size=(64,), fill_value=64, dtype=torch.long),lengths)   
        # calculating gradients
        loss.backward()
        # optimizing
        optimizer.step()
        # .item() is simple way to get value from scalar
        loss_avg += loss.item()
        t+=1
        if i%200==199:
            print('LOSS',loss_avg/(t), e,i,word[0])
            loss_avg=0
            t=0
            break

bad img
LOSS 1.3779514861106872 0 199 in
bad img
LOSS 1.332351380586624 1 199 in
bad img
LOSS 1.2943692576885224 2 199 in
bad img
LOSS 1.2530533784627915 3 199 in
bad img
LOSS 1.213676399886608 4 199 in
bad img
LOSS 1.1733385920524597 5 199 in
bad img


KeyboardInterrupt: 

In [44]:
model.to(torch.device('cpu'))
w=model(img.cpu(),torch.tensor([0]),10)

In [51]:
target[:10,:10]

tensor([[32, 42, 39,  1,  2,  2,  2,  2,  2,  2],
        [11, 47, 49, 39, 21, 37, 40, 39, 37, 32],
        [38, 47, 22, 41, 25,  1,  2,  2,  2,  2],
        [37, 47, 32,  1,  2,  2,  2,  2,  2,  2],
        [14, 39,  1,  2,  2,  2,  2,  2,  2,  2],
        [ 5, 33, 21, 32, 16,  1,  2,  2,  2,  2],
        [32, 47,  1,  2,  2,  2,  2,  2,  2,  2],
        [33, 38, 38, 39,  5, 32, 12, 37, 11,  1],
        [33, 23,  1,  2,  2,  2,  2,  2,  2,  2],
        [33,  1,  2,  2,  2,  2,  2,  2,  2,  2]], device='cuda:0')

In [52]:
for i in range(10):
    qe=w[:,i,:].exp().argmax(dim=1)
    print(qe[qe!=0])

tensor([32, 42, 42, 39, 39,  1,  1])
tensor([11, 47, 39, 40, 37, 40, 39, 32, 32,  1,  1])
tensor([38, 37, 37, 41, 25,  1,  1])
tensor([37, 47, 32, 32,  1,  1])
tensor([14, 39, 39,  1,  1])
tensor([ 5, 33, 33, 16,  1,  1])
tensor([32, 47,  1,  1])
tensor([33, 39, 32, 32, 12, 37, 11,  1,  1])
tensor([33, 23,  1,  1])
tensor([33,  1,  1])
